### Homework 07: Concurrency

## Due Date: Apr 5, 2021, 04:00pm

#### Firstname Lastname: Yuhan Liu

#### E-mail: yl7576@nyu.edu

#### Enter your solutions and submit this notebook


---

**Problem 1** **(60 Points)**

Let us consider the Gamma function, or the Euler integral of the second kind: 

$$\Gamma(x) = \int_{0} ^ \infty t ^{x - 1} e^{-t} dt, $$

and in this HW we consider real $x > 0$.

(Here is more on the Gamma function https://en.wikipedia.org/wiki/Gamma_function .
It is not needed for this HW assignment.) 

## 1.1 (Points 15) 

Write a function (in the cell below) that sequentially calculates the given Gamma integral.


In [149]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from time import time
from functools import partial
from multiprocess import Pool
import logging
import requests

In [150]:
def calculate_gamma(x, bound_1, bound_2, number_of_steps):
    # sequential version to calculate Gamma(x):
    # where we approximate the given integral,
    # like this a discrete sum in number_of_steps
    # equidistant points on the interval [bound_1, bound_2]
    
    # return Gamma(x)
    delta = (bound_2-bound_1)/number_of_steps
    res = 0
    for i in range(0, number_of_steps):
        t = bound_1 + delta * i
        res = res + delta * (t**(x-1))*np.exp(-t)
    return res

## 1.2 (Points 5)

Evaluate, $\Gamma(6)$ by using `calculate_gamma(x, bound_1, bound_2, number_of_steps)` and the error of this computation.


As arguments, use `x=6, bound_1=0, bound_2=1000, number_of_steps=10_000_000`. We know that $\Gamma(x) = x!$, so $\Gamma(6) = 5! = 120$. 


In [129]:
x = 6 
bound_1 = 0
bound_2 = 1000
number_of_steps = 10_000_000

In [130]:
st = time()
gamma_6 = calculate_gamma(x, bound_1, bound_2, number_of_steps)
et = time()
error = 120-gamma_6
print(gamma_6)
print('the error of this computation is',error)
print('the runtime is',et-st,'s')

119.99999999994276
the error of this computation is 5.724132279283367e-11
the runtime is 12.967557907104492 s


---

Write two functions to calculate $\Gamma(x)$ by using:



**1.3.1 (Points 15)**
**threading** with N=4 threads; 

**1.3.2 (Points 15)**
**multiprocessing** with N=4 processes. 


**1.3.3 (Points 10)** 
Compare the times of the three versions and write a short explanation of what you are observing.

How does the answer change when N=8 and why?

    

## 1.3.1 

In [132]:
def multi_thread_gamma(x, bound_1,  bound_2, number_of_steps, num_process):
    chunk = [[x, i*(bound_2-bound_1)/num_process, 
                (i+1)*(bound_2-bound_1)/num_process, 
                int(number_of_steps/num_process)] for i in range(num_process)]
    st = time()
    with ThreadPoolExecutor(max_workers=num_process) as ex:
        ex.map(calculate_gamma, chunk)
    print(sum(res),'with runtime = ', time()-st)

## 1.3.2

In [137]:
def multi_process_gamma(x, bound_1,  bound_2, number_of_steps, num_process):
    chunk = [[x, i*(bound_2-bound_1)/num_process, 
                (i+1)*(bound_2-bound_1)/num_process, 
                int(number_of_steps/num_process)] for i in range(num_process)]
    st = time()
    with Pool(num_process) as pool:
        res = pool.starmap(calculate_gamma, chunk)
    print(sum(res),'with runtime = ', time()-st)

## 1.3.3

In [147]:
multi_thread_gamma(x, bound_1,  bound_2, number_of_steps, num_process=4)

119.99999999994276 with runtime =  6.8755810260772705


In [143]:
multi_process_gamma(x, bound_1, bound_2, number_of_steps, num_process=4)

119.99998799977556 with runtime =  0.004274845123291016


In [148]:
multi_thread_gamma(x, bound_1,  bound_2, number_of_steps, num_process=8)

119.99999999994276 with runtime =  6.678266763687134


In [141]:
multi_process_gamma(x, bound_1,  bound_2, number_of_steps, num_process=8)

119.99998799977556 with runtime =  0.0038111209869384766


## Summary

The runtime of original version is 12.9675 s

The runtime of threading with N = 4 is 6.8756 s

The runtime of threading with N = 8 is 6.6782 s

The runtime of multiprocessing with N = 4 is 0.0043s

The runtime of multiprocessing with N = 8 is 0.0038 s


In this problem, the task is CPU bounded, so multiprocessing is the fastest method with more processes. Threading does not work as well as multiprocessing because Python has a GIL, which allows only one thread to be executed at a time throughout this process, this code is concurrent but not parallel. 

Also, changing from N=4 to N=8 also improves the work because we are using more cores to do the parallel computing.

---

**Problem 2 (40 points)**

__Website uptime__ is the time that a website or web service is available to the users over a given period.

The task is to build an application that checks the uptime of websites. 

- The application should go over a list of website URLs and checks if those websites are up.
- Instead of performing a classic HTTP GET request, it performs a HEAD request so that it does not affect traffic significantly.
- If the HTTP status is in the danger ranges (400+, 500+), a message is casted. 

Here are some useful functions:

In [114]:
#### _website uptimer_ ####
class WebsiteDownException(Exception):
    pass
 
def ping_website(address, timeout=20):
    """
    Check if a website is down. A website is considered down 
    if either the status_code >= 400 or if the timeout expires
     
    Throw a WebsiteDownException if any of the website down conditions are met
    """
    try:
        response = requests.head(address, timeout=timeout)
        if response.status_code >= 400:
            logging.warning("Website %s returned status_code=%s" % (address, response.status_code))
            raise WebsiteDownException()
    except requests.exceptions.RequestException:
        logging.warning("Timeout expired for website %s" % address)
        raise WebsiteDownException()
         
def check_website(address):
    """
    Utility function: check if a website is down, if so, notify the user
    """
    try:
        ping_website(address)
    except WebsiteDownException:
        print('The websie ' + address + ' is down')

---

You need a website list to try our system out. Create your own list or use the following one. 

---

In [115]:
WEBSITE_LIST = [
    'http://amazon.co.uk',
    'http://amazon.com',
    'http://facebook.com',
    'http://google.com',
    'http://google.fr',
    'http://google.es',
    'http://google.co.uk',
    'http://gmail.com',
    'http://stackoverflow.com',
    'http://github.com',
    'http://heroku.com',
    'http://really-cool-available-domain.com',
    'http://djangoproject.com',
    'http://rubyonrails.org',
    'http://basecamp.com',
    'http://trello.com',
    'http://shopify.com',
    'http://another-really-interesting-domain.co',
    'http://airbnb.com',
    'http://instagram.com',
    'http://snapchat.com',
    'http://youtube.com',
    'http://baidu.com',
    'http://yahoo.com',
    'http://live.com',
    'http://linkedin.com',
    'http://netflix.com',
    'http://wordpress.com',
    'http://bing.com',
]

---

A serial version of the _website uptimer_ can be written as: 

---


In [116]:
start_time = time.time()
 
for address in WEBSITE_LIST:
    check_website(address)
         
end_time = time.time()        
 
print("Time for Serial: %ssecs" % (end_time - start_time))

The websie http://really-cool-available-domain.com is down


The websie http://another-really-interesting-domain.co is down
Time for Serial: 7.73346471786499secs


You should build two versions of the **website uptimer**, by using:

**2.1 (Points 15)**
**threading** with N=4 threads; 

**2.2 (Points 15)**
**multiprocessing** with N=4 processes. 

**2.3 (Points 10)** 

Compare the times of the three versions and write a short explanation of what you are observing.

How does the answer change when N=8 and why?


## 2.1

In [117]:
num_process = 4

In [124]:
start = time()
with ThreadPoolExecutor(max_workers=num_process) as ex:
        ex.map(check_website, WEBSITE_LIST)   
end = time()  
print('runtime is',end-start, ' s')

The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
runtime is 1.5776216983795166  s


## 2.2

In [123]:
start = time()
with Pool(num_process) as p:
     results = p.map(check_website, WEBSITE_LIST)      
end = time()  
print('runtime is',end-start , 's')

The websie http://really-cool-available-domain.com is down


The websie http://another-really-interesting-domain.co is down
runtime is 1.651716947555542 s


## 2.3

### N = 8

In [146]:
num_process = 8

start = time()
with ThreadPoolExecutor(max_workers=num_process) as ex:
        ex.map(check_website, WEBSITE_LIST)   
end = time()  
print('runtime of threading with N = 8 is',end-start, ' s')

start = time()
with Pool(num_process) as p:
     results = p.map(check_website, WEBSITE_LIST)      
end = time()  
print('runtime of multiplprocessing with N = 8 is',end-start , 's')

The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
runtime of threading with N = 8 is 0.9660801887512207  s


The websie http://really-cool-available-domain.com is down


The websie http://another-really-interesting-domain.co is down
runtime of multiplprocessing with N = 8 is 1.0014081001281738 s


### Summary

The runtime of original version is 7.7335 s

The runtime of threading with N = 4 is 1.5776 s

The runtime of threading with N = 8 is 0.9660 s

The runtime of multiplprocessing with N = 4 is 1.6517 s

The runtime of multiplprocessing with N = 8 is 1.0014 s

In this case, the task is I/O bound, so the majority of the time is spent waiting for the network. The thread allows us to go to the next request when waiting for the response. This is why threading can provide a slight speed increase than multiprocessing method. 

Also, changing from 4 to 8 processes or threads can also improve the work because more concurrency is achieved.